In [1]:
import pandas as pd

In [2]:
db_df = pd.read_csv('drugbank_filtered.csv')
cs_df = pd.read_csv('EGFR_kiba.csv')
AR_df = pd.read_csv('drugbank_EGFR.csv')
db_df.shape, cs_df.shape, AR_df.shape

((10568, 3), (10568, 11), (31, 5))

In [3]:
cs_df.head()

,dataset,y_true,y_pred,view1,entity1,top_segments1,view2,entity2,sequence,top_segments2,top_residues
0,case_study,0.0,0.511303,gconv,C[C@H]1O[C@H](O[C@@H]2[C@@H](CO)O[C@H](O[C@@H]...,"[['C', 21], ['C', 50], ['C', 52], ['O', 28], [...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['EDHFLSLQRMFNN', 'PPICTIDVYMIMV', 'TSNKLTQLGT...",['Glu45 Asp46 His47 Phe48 Leu49 Ser50 Leu51 Gl...
1,case_study,0.0,0.435567,gconv,CN1CCC(CC1)=C1C2=NC=C(C=O)N2CCC2=CC=CC=C12,"[['O', 13], ['N', 9], ['C', 0], ['N', 1], ['C'...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
2,case_study,0.0,0.636804,gconv,C#CCN[C@@H]1CCC2=CC=CC=C12,"[['C', 5], ['N', 3], ['C', 0], ['C', 1], ['C',...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
3,case_study,0.0,0.469631,gconv,OS(=O)(=O)C1=CC2=C(N\C(C2=O)=C2\NC3=C(C=C(C=C3...,"[['O', 22], ['C', 19], ['O', 21], ['C', 17], [...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...
4,case_study,0.0,0.523893,gconv,CC1=C(O)C=CC=C1C(=O)NC1=C2C=C(C=CC2=CC=C1)C(N)=N,"[['O', 9], ['C', 8], ['C', 21], ['C', 6], ['C'...",rnn,EGFR,MRPSGTAGAALLALLAALCPASRALEEKKVCQGTSNKLTQLGTFED...,"['ISGNKNLCYANTI', 'FRDEATCKDTCPP', 'ENSECIQCHP...",['Ile463 Ser464 Gly465 Asn466 Lys467 Asn468 Le...


In [4]:
predictions = {'id':[],'drug':[],'target':[], 'value':[], 'drugbank?':[]}
for idx, row in cs_df.iterrows():
    cs_smiles = row['entity1']
    db_row = db_df[db_df['smiles']==cs_smiles]
    db_id = db_row['drugbank_id'].iloc[0]
    predictions['id'].append(db_id)
    predictions['drug'].append(db_row['name'].iloc[0])
    predictions['target'].append(row['entity2'])
    predictions['value'].append(row['y_pred'])
    s = AR_df[AR_df['DRUGBANK ID']==db_id]
    if len(s) > 0:
        predictions['drugbank?'].append('yes')
    else:
        predictions['drugbank?'].append('no')    

In [5]:
predictions = pd.DataFrame(predictions)
predictions = predictions.sort_values('value', ascending=True)
predictions.head(30)

,id,drug,target,value,drugbank?
7947,DB05424,Canertinib,EGFR,-1.946079,yes
2400,DB08916,Afatinib,EGFR,-1.756241,yes
6638,DB12174,CUDC-101,EGFR,-1.742409,no
6679,DB07662,N-[4-(3-BROMO-PHENYLAMINO)-QUINAZOLIN-6-YL]-AC...,EGFR,-1.725297,yes
6107,DB00317,Gefitinib,EGFR,-1.623227,yes
6417,DB11963,Dacomitinib,EGFR,-1.612542,yes
8459,DB08564,(2E)-N-{4-[(3-bromophenyl)amino]quinazolin-6-y...,EGFR,-1.563200,no
7800,DB00530,Erlotinib,EGFR,-1.541901,yes
2039,DB03365,"4-[3-Hydroxyanilino]-6,7-Dimethoxyquinazoline",EGFR,-1.505018,no
7464,DB06021,AV-412,EGFR,-1.439492,yes


In [47]:
predictions.to_csv('predictions_EGFR_davis.csv', index=False)